In [49]:
from core.ConfigConnection import ConfigConnection
from core.Backup import Backup
from core.Tools import Tools

import mysql.connector

if __name__ == "__main__":
    config = ConfigConnection(path="./config.ini")
    connection = mysql.connector.connect(**config.getConfig())
    cursor = connection.cursor()
    tools = Tools()
    
    sql_file = tools.readFile('SQL', 'dml-web-scraping.sql')
    sql_script = ''.join(sql_file)

    sql_commands = sql_script.split(';')

    cursor = connection.cursor()
    
    try:
        # Ejecuta cada comando SQL individual
        for command in sql_commands:
            try:
                if command.strip():
                    cursor.execute(command)
            except mysql.connector.Error as err:
                print(f"Error executing SQL command: {err}")
           
        print("Script SQL ejecutado exitosamente.")
    except Exception as e:
        print("Error al ejecutar el script SQL:", e)
        connection.rollback()
    finally:
        cursor.close()
        connection.close()

Script SQL ejecutado exitosamente.


In [50]:
import configparser
import mysql.connector

fig = ConfigConnection(path="./config.ini")
connection = mysql.connector.connect(**config.getConfig())
cursor = connection.cursor()

try:
    # Ejecutar la consulta SHOW DATABASES
    cursor.execute("SHOW DATABASES")

    # Recorrer los resultados y mostrar los nombres de las bases de datos
    print("Bases de datos disponibles:")
    for (database,) in cursor:
        print(database)

except Exception as e:
    print("Error al ejecutar la consulta:", e)
finally:
    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

Bases de datos disponibles:
IIDDBDWorldFactbook
IIDDBDsmartwatch
information_schema
mysql
performance_schema
sys


In [53]:
tools = Tools()

json_data = tools.readFileJson("data", "world-factbook.json")

db_config = {
    "host": "localhost",
    "user": "admin",
    "password": "admin",
    "database": "IIDDBDWorldFactbook"
}

connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
cursor.execute('DELETE from Countries;')
cursor.execute('DELETE from Economy;')
cursor.execute('DELETE from Education;')
cursor.execute('DELETE from Health;')
cursor.execute('DELETE from HealthProblems;')

for country, info in json_data.items():
    try:
        # Inserta en la tabla Countries
        cursor.execute("INSERT INTO Countries (tex_name, dec_population, dec_population_growth_percent, dec_area_km_sq, tex_map_references, int_coastline_km_sq, int_max_elevation_km, int_min_elevation_km, dec_population_density) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                       (info['countrie'], 
                        float(info['population']), 
                        float(info['Population growth rate']) if info['Population growth rate'] != '' and info['Population growth rate'] != 'NA' else 0, 
                        float(info['area']) if info['area'] != '' else 0, 
                        info['Map references'] if info['countrie'] != 'france' else 'Europe', 
                        float(info['Coastline']), 
                        float(info['elevation highest point']) if info['elevation highest point'] != '' else 0, 
                        float(info['elevation lowest point']) if info['elevation lowest point'] != '' else 0, 
                        float(info['population']) / float(info['area']) if info['area'] != '' and float(info['area']) != 0 else 0
                       ))
        
        # Obtiene el ID insertado en Countries
        country_id = cursor.lastrowid

        # Inserta en la tabla Economy (usando country_id como llave foránea)
        cursor.execute("INSERT INTO Economy (id_countries_fk, dec_GDP_general_billion_of_dollars, dec_GDP_contribution_agriculture_percent, dec_GDP_contribution_industry_percent, dec_GDP_contribution_service_percent) VALUES (%s, %s, %s, %s, %s)",
                       (country_id, info['economy']['GDP'], info['economy']['GDP_contribution_agriculture'], info['economy']['GDP_contribution_industry'], info['economy']['GDP_contribution_services']))
        
        # Inserta en la tabla Education (usando country_id como llave foránea)
        cursor.execute("INSERT INTO Education (id_countries_fk, dec_literacy_percent, int_school_life_time, dec_education_cost_percent_GDP) VALUES (%s, %s, %s, %s)",
                       (country_id, info['education']['Literacy'], info['education']['school life expectancy'], info['education']['Education expenditures']))
        
        # Inserta en la tabla Health (usando country_id como llave foránea)
        cursor.execute("INSERT INTO Health (id_countries_fk, int_life_expectancy_years, int_infant_mortality_deathover1000births) VALUES (%s, %s, %s)",
                       (country_id, info['health']['life expectancy at birth'], info['health']['infant mortality rate']))

        # Obtiene el ID insertado en Health
        health_id = cursor.lastrowid
        # Inserta en la tabla HealthProblems (usando country_id como llave foránea)
        cursor.execute("INSERT INTO HealthProblems (id_health_fk, dec_health_expenses_percent_GDP, int_physicians_over1000population, int_hospitalbeds_density_bedsover1000population) VALUES (%s, %s, %s, %s)",
                       (health_id, info['health']['Health problems']['Current health expenditure'], info['health']['Health problems']['Physicians density'], info['health']['Health problems']['Hospital bed density']))

        # Realiza commit para cada país
        connection.commit()
    except mysql.connector.Error as err:
        print(f"Error inserting data for {info['countrie']}: {err}")

# Cierra el cursor y la conexión
cursor.close()
connection.close()
print("Se insertaron los datos")

Se insertaron los datos


In [48]:
import mysql.connector

querys = [
    # Población
    # 1. ¿Cuáles son los países más y menos poblados?
    "SELECT tex_name AS Country, dec_population AS Population FROM Countries Where tex_name != 'world' ORDER BY dec_population DESC LIMIT 1;",
    "SELECT tex_name AS Country, dec_population AS Population FROM Countries Where tex_name != 'world' AND dec_population != 0 ORDER BY dec_population ASC LIMIT 1;",
    # 2. ¿Cuál es la densidad de población promedio por región o continente?
    "SELECT tex_map_references AS Continente, AVG(dec_population_density) AS AveragePopulationDensity FROM Countries GROUP BY tex_map_references;",
    # 3¿Cuáles son los países con la tasa de crecimiento demográfico más alta o más baja?
    "SELECT tex_name AS Country, dec_population_growth_percent AS PopulationGrowthPercent FROM Countries ORDER BY dec_population_growth_percent DESC LIMIT 1;",
    "SELECT tex_name AS Country, dec_population_growth_percent AS PopulationGrowthPercent FROM Countries Where dec_population_growth_percent != 0 ORDER BY dec_population_growth_percent ASC LIMIT 1;",
    
    # Economía
    # 1. ¿Cuáles son las principales economías del mundo en términos de PIB?
    "SELECT tex_name AS Country, dec_GDP_general_billion_of_dollars AS GDP FROM Economy JOIN Countries ON Economy.id_countries_fk = Countries.id ORDER BY dec_GDP_general_billion_of_dollars DESC LIMIT 10;",
    # 2. ¿Cuáles son los países con el ingreso per cápita más alto o más bajo?
    "SELECT tex_name AS Country, dec_GDP_general_billion_of_dollars / dec_population AS IncomePerCapita FROM Economy JOIN Countries ON Economy.id_countries_fk = Countries.id Where dec_population != 0 AND dec_GDP_general_billion_of_dollars != 0 ORDER BY IncomePerCapita DESC LIMIT 10;",
    "SELECT tex_name AS Country, dec_GDP_general_billion_of_dollars / dec_population AS IncomePerCapita FROM Economy JOIN Countries ON Economy.id_countries_fk = Countries.id Where dec_population != 0 AND dec_GDP_general_billion_of_dollars != 0 ORDER BY IncomePerCapita ASC LIMIT 10;",
    # 3. ¿Cuáles son los sectores económicos más importantes en diferentes países?
    "SELECT tex_name AS Country, dec_GDP_contribution_agriculture_percent AS Agriculture, dec_GDP_contribution_industry_percent AS Industry, dec_GDP_contribution_service_percent AS Services FROM Economy JOIN Countries ON Economy.id_countries_fk = Countries.id ORDER BY Agriculture DESC, Industry DESC, Services DESC LIMIT 10;",
    
    # Geografía
    # 1. ¿Cuáles son los países más grandes y más pequeños en términos de área?
    "SELECT tex_name AS Country, dec_area_km_sq AS Area FROM Countries WHERE dec_area_km_sq != 0 ORDER BY dec_area_km_sq DESC LIMIT 10;",
    "SELECT tex_name AS Country, dec_area_km_sq AS Area FROM Countries WHERE dec_area_km_sq != 0 ORDER BY dec_area_km_sq ASC LIMIT 10;",
    # 2. ¿Cuál es la longitud de las costas de diferentes países?
    "SELECT tex_name AS Country, int_coastline_km_sq AS CoastlineLength FROM Countries ORDER BY int_coastline_km_sq DESC;",
    # 3. ¿Cuáles son los países con la altitud más alta o más baja?
    "SELECT tex_name AS Country, int_max_elevation_km AS MaxElevation FROM Countries ORDER BY int_max_elevation_km DESC LIMIT 10;",
    "SELECT tex_name AS Country, int_min_elevation_km AS MinElevation FROM Countries ORDER BY int_min_elevation_km ASC LIMIT 10;",
    
    # Educación
    # 1. ¿Cuáles son los países con los niveles más altos de alfabetización?
    "SELECT tex_name AS Country, dec_literacy_percent AS LiteracyPercent FROM Education JOIN Countries ON Education.id_countries_fk = Countries.id ORDER BY dec_literacy_percent DESC LIMIT 10;",
    # 2. ¿Cuál es la tasa de matriculación escolar en diferentes países?
    "SELECT tex_name AS Country, int_school_life_time AS SchoolLifeTime FROM Education JOIN Countries ON Education.id_countries_fk = Countries.id;",
    # 3. ¿Cuáles son los países con la mayor inversión en educación?
    "SELECT tex_name AS Country, dec_education_cost_percent_GDP AS EducationInvestmentPercent FROM Education JOIN Countries ON Education.id_countries_fk = Countries.id ORDER BY dec_education_cost_percent_GDP DESC LIMIT 10;",
    
    # Salud
    # 1. ¿Cuál es la esperanza de vida promedio en diferentes países?
    "SELECT tex_name AS Country, int_life_expectancy_years AS LifeExpectancy FROM Health JOIN Countries ON Health.id_countries_fk = Countries.id;",
    # 2. ¿Cuáles son los países con la tasa de mortalidad infantil más alta o más baja?
    "SELECT tex_name AS Country, int_infant_mortality_deathover1000births AS InfantMortalityRate FROM Health JOIN Countries ON Health.id_countries_fk = Countries.id ORDER BY int_infant_mortality_deathover1000births DESC LIMIT 10;",
    "SELECT tex_name AS Country, int_infant_mortality_deathover1000births AS InfantMortalityRate FROM Health JOIN Countries ON Health.id_countries_fk = Countries.id ORDER BY int_infant_mortality_deathover1000births ASC LIMIT 10;",
    # 3. ¿Cuáles son los principales problemas de salud en diferentes regiones?
    "SELECT tex_name AS Country, dec_health_expenses_percent_GDP AS HealthExpensesPercent, int_physicians_over1000population AS PhysiciansPer1000People, int_hospitalbeds_density_bedsover1000population AS HospitalBedsPer1000People FROM HealthProblems JOIN Health ON HealthProblems.id_health_fk = Health.id JOIN Countries ON Health.id_countries_fk = Countries.id;"
]

db_config = {
    "host": "localhost",
    "user": "admin",
    "password": "admin",
    "database": "IIDDBDWorldFactbook"
}

try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()

    cursor.execute(querys[10])
    result = cursor.fetchall()  # Fetch all rows of the result

    for row in result:
        print(row[0])
        print(row[1])
        #print(row[2])
        #print(row[3])
        #print("Area:", row[2])
        print()
        
except mysql.connector.Error as err:
    print("Error:", err)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()


niger
1.27

chad
1.28

monaco
2.00

coral sea islands
3.00

navassa island
5.00

ashmore and cartier islands
5.00

clipperton island
6.00

gibraltar
7.00

wake island
7.00

paracel islands
8.00

